CIFAR

In [1]:
import torch
import os
import opacus
import copy
import torchvision
import random
import torchattacks

import numpy as np
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim.lr_scheduler as lr_scheduler

from tqdm import tqdm
from datetime import datetime
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager

from model import ResNet9, initialize_weights
from adv import Attack, adv_test

seed = 34
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.deterministic = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

ImportError: cannot import name 'initialize_weights' from 'model' (/home/anna/application_proj/adv_dpsgd/src/model.py)

In [ ]:
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

class Experiment_weighted:
    def __init__(self, batch_size, epochs, adv_attack, adv_attack_mode, adv_params, device, save_experiment,
                 verbose, dp = True, target_epsilon=7.5, dataset='cifar', adv_test=True, name=None, max_batch_size=512):
        if dataset == 'cifar':
            self.model = ResNet9(norm_layer="group")
            self.adv_model = ResNet9(norm_layer="group").to(device)
        else:
            self.model = ResNet9(in_channels=1, norm_layer="group")
            self.adv_model = ResNet9(in_channels=1, norm_layer="group").to(device)
        self.optimizer = optim.NAdam(self.model.parameters())
        self.lr_scheduler = lr_scheduler.ReduceLROnPlateau(self.optimizer)
        self.criterion = nn.CrossEntropyLoss()
        self.batch_size = batch_size
        self.epochs = epochs
        self.attack = Attack(adv_attack, adv_params, device) if adv_attack is not None else None
        self.adv_attack_mode = adv_attack_mode
        self.dp = dp
        self.device = device
        self.verbose = verbose
        self.disable_tqdm = not self.verbose
        self.save_experiment = save_experiment
        self.target_epsilon = target_epsilon
        self.dataset = dataset
        self.adv_test = adv_test
        self.max_batch_size = max_batch_size
        self.best_model_weights = None


        if name is None:
            adv_s = f"adv-{adv_attack_mode}" if adv_attack is not None else "non_adv"
            dp_s = "dp" if dp else "non_dp"
            self.name = f"{dataset}+{adv_s}+{dp_s}+{batch_size}"
        else:
            self.name = name

        now = datetime.now()
        formatted_timestamp = now.strftime("%d-%m-%Y_%H:%M:%S")
        self.dir_name = f"results/{self.name}_{formatted_timestamp}"
        if self.save_experiment:
            os.makedirs(self.dir_name, exist_ok=True)

        self._setup_training()

    def _log(self, message):
        print(f"Experiment {self.name}: {message}")

    def _setup_training(self):
        self._log("Loading data")
        if self.dataset == 'cifar':
            transform = transforms.Compose(
                [transforms.ToTensor(),
                transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV) 
                ])
            learning_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

            # train_set, val_set = torch.utils.data.random_split(learning_set, [35000, 15000],
            #                                                 generator=torch.Generator().manual_seed(42))
            test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
            self.classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        else:
            transform = transforms.Compose(
                [transforms.ToTensor(),
                ])
            learning_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
            # train_set, val_set = torch.utils.data.random_split(learning_set, [40000, 20000],
            #                                                generator=torch.Generator().manual_seed(42))
            test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
            self.classes = (x for x in range(0,9))

        self.train_loader = torch.utils.data.DataLoader(learning_set, batch_size=self.batch_size, shuffle=True,
                                                        num_workers=2)
        # self.val_loader = torch.utils.data.DataLoader(val_set, batch_size=self.batch_size, num_workers=2)
        # self.val_loader_x1 = torch.utils.data.DataLoader(val_set, batch_size=1, num_workers=2)
        self.test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)


        if self.dp:
            self._log("DP on")
            self.privacy_engine = opacus.PrivacyEngine()
            self.model, self.optimizer, self.train_loader = self.privacy_engine.make_private_with_epsilon(
                module=self.model,
                optimizer=self.optimizer,
                data_loader=self.train_loader,
                epochs=self.epochs,
                target_epsilon=self.target_epsilon,
                target_delta=1e-5,
                max_grad_norm=1.2,
                noise_generator=torch.Generator(device=self.device).manual_seed(seed)
            )
            self.lr_scheduler = lr_scheduler.ReduceLROnPlateau(self.optimizer)
        else:
            self._log("DP off")

    def _fit(self):
        self._log("Training started")
        _ = self.model.to(self.device)
        _ = self.model.apply(initialize_weights)

        train_losses = []
        train_accuracies = []
        val_losses = []
        val_accuracies = []

        for epoch in range(self.epochs):

            if self.dp:
                with BatchMemoryManager(
                        data_loader=self.train_loader,
                        max_physical_batch_size=self.max_batch_size,
                        optimizer=self.optimizer
                ) as new_train_loader:
                    train_loss, train_acc = self._run_epoch(new_train_loader)
            else:
                train_loss, train_acc = self._run_epoch(self.train_loader)

            self._log(f"Epoch {epoch + 1: >3}/{self.epochs}, train loss: {train_loss:.2e}, train acc: {train_acc:.3f}")

            train_losses.append(train_loss)
            train_accuracies.append(train_acc)
#             self.lr_scheduler.step(train_loss)
            
            val_loss, val_acc = self._run_epoch(self.test_loader, eval=True)
            self.lr_scheduler.step(val_loss)
            
            self._log(f"Epoch {epoch + 1: >3}/{self.epochs}, val loss: {val_loss:.2e}, val acc: {val_acc:.3f}")
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)

        self._plot(range(len(train_losses)), np.array(train_losses), 'train_loss')
        self._plot(range(len(train_accuracies)), np.array(train_accuracies), 'train_acc')
        self._plot(range(len(val_losses)), np.array(val_losses), 'val_loss')
        self._plot(range(len(val_accuracies)), np.array(val_accuracies), 'val_acc')

    def _plot(self, x, y, title):
        plt.plot(range(len(x)), np.array(y))
        plt.title(title)
        if self.save_experiment:
            plt.savefig(f"{self.dir_name}/{title}.png")
        if self.verbose:
            plt.show()
        else:
            plt.clf()

    def _run_epoch(self, data_loader, eval=False):
        if eval:
            self.model.eval()
        else:
            self.model.train()

        dataset_len = len(data_loader.dataset)

        epoch_loss = 0.0
        epoch_acc = 0.0

        for inputs, targets in tqdm(data_loader, disable=self.disable_tqdm):
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            if self.dp:
                self.adv_model.load_state_dict(copy.deepcopy(self.model._module.state_dict()))
                model = self.adv_model
            else:
                model = self.model

            inputs_adv, _ = self.attack.perturbe(inputs, targets, model, self.adv_attack_mode)

            self.optimizer.zero_grad()

            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)

            top1 = torch.argmax(outputs, dim=1).long()
            n_correct = torch.sum(top1 == targets)
            epoch_loss += loss.item()
            epoch_acc += n_correct.item()
            
            outputs_adv = self.model(inputs_adv)
            loss_adv = self.criterion(outputs_adv, targets)
            
            loss_weighted = (loss * 9 + loss_adv) / 10
#             loss_weighted = loss + loss_adv

            if not eval:
                loss_weighted.backward()
                self.optimizer.step()

        epoch_loss /= dataset_len
        epoch_acc /= dataset_len

        return epoch_loss, epoch_acc

    def run(self):
        self._fit()

        self._log(f"Train accuracy: {self._run_epoch(self.train_loader, eval=True)}")
        epsilon = self.privacy_engine.get_epsilon(1e-5)
        self._log(f"Training finished with epsilon {epsilon}")
        
        # if self.adv_test:
        #     adv_test(self.model, self.val_loader_x1, self.classes, self.device, self.attack)

       

In [ ]:
#vanilla dp
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
from experiment import Experiment
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=50, 
    adv_attack=None, 
    adv_attack_mode=None, 
    adv_params=None, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=8, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=256)
ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#vanilla dp
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print(device)
# from experiment import Experiment
# ex = Experiment_weighted(
#     batch_size=1024, 
#     epochs=50, 
#     adv_attack=None, 
#     adv_attack_mode=None, 
#     adv_params=None, 
#     device=device, 
#     save_experiment=False, 
#     verbose=True, 
#     target_epsilon=7.5, 
#     dataset='cifar', 
#     adv_test=True, 
#     max_batch_size=256)
# ex.run()

In [ ]:
# ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#vanilla dp + adv
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print(device)
# from experiment import Experiment
# ex = Experiment(
#     batch_size=1024, 
#     epochs=50, 
#     adv_attack=torchattacks.FFGSM, 
#     adv_attack_mode='batch', 
#     adv_params={'eps':8/255}, 
#     dp=True, 
#     device=device, 
#     save_experiment=False, 
#     verbose=True, 
#     target_epsilon=2.5, 
#     dataset='cifar', 
#     adv_test=True, 
#     max_batch_size=256)

# ex.run()

In [ ]:
# ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
# #vanilla dp + adv
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print(device)
# from experiment import Experiment
# ex = Experiment(
#     batch_size=1024, 
#     epochs=50, 
#     adv_attack=torchattacks.PGD, 
#     adv_attack_mode='batch', 
#     adv_params={'eps':8/255, 'alpha':1/255, 'steps':5, 'random_start':True}, 
#     dp=True, 
#     device=device, 
#     save_experiment=False, 
#     verbose=True, 
#     target_epsilon=2.5, 
#     dataset='cifar', 
#     adv_test=True, 
#     max_batch_size=256)

# ex.run()

In [ ]:
# ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#90 clean - 10 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=50, 
    adv_attack=torchattacks.FFGSM, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=8, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=256)

ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#90 clean - 10 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=50, 
    adv_attack=torchattacks.FFGSM, 
    adv_attack_mode='sample', 
    adv_params={'eps':8/255}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=256)

ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
# 50 clean -50 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=50, 
    adv_attack=torchattacks.FFGSM, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=256)

ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#90 clean - 10 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=50, 
    adv_attack=torchattacks.PGD, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255, 'alpha':1/255, 'steps':5, 'random_start':True}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=64)
ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#90 clean - 10 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=50, 
    adv_attack=torchattacks.PGD, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255, 'alpha':1/255, 'steps':5, 'random_start':True}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=128)
ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#90 clean - 10 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=25, 
    adv_attack=torchattacks.FFGSM, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=128)

ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#90 clean - 10 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=25, 
    adv_attack=torchattacks.PGD, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255, 'alpha':1/255, 'steps':5, 'random_start':True}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=128)
ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#80 clean - 20 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=25, 
    adv_attack=torchattacks.PGD, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255, 'alpha':1/255, 'steps':5, 'random_start':True}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=128)
ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)

In [ ]:
#80 clean - 20 adv
torch.cuda.empty_cache() 
ex = Experiment_weighted(
    batch_size=1024, 
    epochs=25, 
    adv_attack=torchattacks.PGD, 
    adv_attack_mode='batch', 
    adv_params={'eps':8/255, 'alpha':1/255, 'steps':5, 'random_start':True}, 
    dp=True, 
    device=device, 
    save_experiment=False, 
    verbose=True, 
    target_epsilon=2.5, 
    dataset='cifar', 
    adv_test=True, 
    max_batch_size=128)
ex.run()

In [ ]:
ex._run_epoch(ex.test_loader, eval=True)